In [ ]:
%pip install stable-baselines3

### Train agent

In [ ]:
import gymnasium as gym
from stable_baselines3 import DDPG, SAC, HerReplayBuffer

import panda_gym

env = gym.make("PandaPickAndPlace", render_mode="human")

model = SAC(
    policy="MultiInputPolicy", 
    env=env, 
    replay_buffer_class=HerReplayBuffer, 
    verbose=1, 
    buffer_size=1000000,
    replay_buffer_kwargs=dict(
        n_sampled_goal=4,
        goal_selection_strategy='future',
    ),
    gamma=0.95,
    learning_starts=1000,
    train_freq=1
    )

model.learn(total_timesteps=4000000)
model.save("./her_robot")

### Evaluate

In [ ]:
import gymnasium as gym
from stable_baselines3 import SAC, HerReplayBuffer
import panda_gym
from time import sleep

env = gym.make("PandaPickAndPlace", render_mode="human")

model = SAC.load("./her_robot", env=env)
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(10000):
    sleep(0.08)
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    vec_env.render()

    if done:
       obs = vec_env.reset()